# Model group generators

Model groups can be configured with object placement engines and spatial constraints to generate a group of models to be added to a `ModelGroup` set. 

`ModelGroup` entities can keep a group of models under an unique frame and easily spawn the models with respect to the model group frame. 
The model group can then be spawned in Gazebo as **multiple individual models** or **nested models**.

In this example, a crate is created from a mesh and filled with **spheres**, **cuboids** and **cylinders** of random dimensions and masses.

In [1]:
import os
from pcg_gazebo.generators import ModelGroupGenerator

generator = ModelGroupGenerator('full_crate')

Add the single-link model of a crate generated from the mesh below to the assets manager.

![crate](images/crate.png)

In [2]:
generator.add_asset(
    tag='crate',
    description=dict(
        type='mesh',
        args=dict(
            visual_mesh='file://' + os.path.abspath('meshes/crate.stl'),
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=False,
            name='crate',
            color='xkcd'
        )
    )
)

True

Add **spheres**, **cuboids** and **cylinder** models to the assets manager to fill the crate.

Since the dimensions and masses are defined by **lambda functions**, the respective **model factory** for the model primitives is going to be called and the lambda functions run to set the parameters of the model.

In [3]:
generator.add_asset(
    tag='ball',
    description=dict(
        type='sphere',
        args=dict(
            radius="max(0.05, 0.3 * __import__('numpy').random.random())",
            name='sphere',
            mass="max(0.1, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='cuboid',
    description=dict(
        type='box',
        args=dict(
            size="0.5 * __import__('numpy').random.random(3)",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='cylinder',
    description=dict(
        type='cylinder',
        args=dict(
            length="max(0.05, 0.2 *__import__('numpy').random.random())",
            radius="max(0.05, 0.2 *__import__('numpy').random.random())",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

True

Add constraint to place objects tangent to the ground plane.

In [4]:
generator.add_constraint(
    name='tangent_to_ground_plane',
    type='tangent',
    frame='world',
    reference=dict(
        type='plane',
        args=dict(
            origin=[0, 0, 0],
            normal=[0, 0, 1]
        )
    )
)

True

Add a workspace above the crate where the filling objects should be spawned on top of the crate.

In [5]:
generator.add_constraint(
    name='crate_base',
    type='workspace',
    frame='world',
    geometry=dict( 
        type='area',
        description=dict(
          points=[ 
              [-0.5, -0.4, 0],
              [-0.5, 0.4, 0],
              [0.5, 0.4, 0],
              [0.5, -0.4, 0]
          ]
        )
  )
)

True

Add a fixed-pose engine to place the crate on the ground plane.

In [6]:
generator.add_engine(
    tag='crate_engine',
    engine_name='fixed_pose',
    models=['crate'],
    poses=[
        [0, 0, 0, 0, 0, 0]
    ],
    constraints=[
        dict(
            model='crate',
            constraint='tangent_to_ground_plane'
        )
    ]
)

True

Add engine to randomily spawn the spheres, cuboids and cylinder above the crate. 

A collision checker makes sure none of the objects overlap or are within another.

In [7]:
generator.add_engine(
    tag='fill_crate',
    engine_name='random_pose',
    models=['ball', 'cuboid', 'cylinder'],
    max_num=dict(
        ball=4,
        cuboid=4,
        cylinder=4
    ),
    model_picker='random',
    no_collision=True,
    policies=[
        dict(
            models=['ball', 'cuboid', 'cylinder'],
            config=[
                dict(
                    dofs=['x', 'y'],
                    policy=dict(
                        name='workspace',
                        args='crate_base'
                    )
                ),
                dict(
                    dofs=['z'],
                    policy=dict(
                        name='uniform',
                        args=dict(
                            mean=0.5,
                            min=0.0,
                            max=3.0
                        )
                    )
                ),
                dict(
                    dofs=['roll', 'pitch', 'yaw'],
                    policy=dict(
                        name='uniform',
                        args=dict(
                            mean=0,
                            min=-3.141592653589793,
                            max=3.141592653589793
                        )
                    )
                )
            ]
        )
    ]
)

True

Every time the model generator is run, all the assets that have dynamic parameters are re-generated. This makes it possible to have a single configuration for multiple model groups.

In [8]:
# Generate model groups
models = list()
for i in range(3):
    models.append(generator.run(group_name='full_crate_{}'.format(i)))

Start an instance of Gazebo

In [9]:
from pcg_gazebo.generators.creators import create_models_from_config
from pcg_gazebo.task_manager import Server

# Start an empty world Gazebo simulation
server = Server()
server.create_simulation('default')
simulation = server.get_simulation('default')
simulation.create_gazebo_empty_world_task()
print(simulation.get_task_list())
print('Is Gazebo running: {}'.format(
    simulation.is_task_running('gazebo')))
simulation.run_all_tasks()

# Create a Gazebo proxy
gazebo_proxy = simulation.get_gazebo_proxy()

# Pause the simulation
gazebo_proxy.pause()

2020-01-30 18:24:42,739 | ERROR | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
['gazebo']
Is Gazebo running: False
2020-01-30 18:24:43,076 | ERROR | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2020-01-30 18:24:43,082 | ERROR | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2020-01-30 18:24:43,412 | ERROR | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.
2020-01-30 18:24:43,734 | ERROR | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1.


Spawn model groups as nested models.

In [10]:
# Spawn models
for i in range(len(models)):
    models[i].spawn(robot_namespace='full_crate_{}'.format(i), gazebo_proxy=gazebo_proxy, pos=[i * 2 - 2, 0, 0], nested=True)

In [12]:
simulation.kill_all_tasks()

A sample of the generated full crates can be seen below

![sim_generated_crates](images/sim_generated_crates.png)